Mudanças:
*Na relação filmes, adicionamos a coluna title, runtime, popularity, status, revenue, release_date

In [223]:
import pandas as pd
from sqlalchemy import create_engine
import pymysql

# Relação filmes

In [224]:
movies_metadata = pd.read_csv('moviesdt/movies_metadata.csv')
credits = pd.read_csv('moviesdt/credits.csv')
ratings = pd.read_csv('moviesdt/ratings.csv')
keywords = pd.read_csv('moviesdt/keywords.csv')

KeyboardInterrupt: 

In [ ]:
filmes = movies_metadata

### Na relação filmes, mudar os atributos para os listados abaixo

In [ ]:
colunas_filmes = ['id', 'homepage', 'adult', 'budget', 'original_language', 'original_title', 'overview', 'id_coletanea', 'title', 'runtime', 'popularity', 'revenue', 'release_date', 'status'] #colunas que teremos no novo dataset e não precisam ser tratadas

In [ ]:
list(set(filmes.columns) - set(colunas_filmes))

['genres',
 'vote_average',
 'belongs_to_collection',
 'production_companies',
 'vote_count',
 'tagline',
 'poster_path',
 'video',
 'production_countries',
 'imdb_id',
 'spoken_languages']

In [ ]:
for coluna in list(set(filmes.columns) - set(colunas_filmes)):
        filmes = filmes.drop(coluna, axis = 1)

In [ ]:
filmes['id_coletanea'] = movies_metadata['belongs_to_collection']
coletIds = filmes['id_coletanea']

In [ ]:
filmes = filmes.reindex(columns=['id', 'title', 'homepage', 'adult', 'budget', 'original_language', 'original_title', 'overview', 'id_coletanea', 'runtime', 'popularity', 'revenue', 'release_date', 'status'])

In [ ]:
ids_colet = []

for coletanea in coletIds:
    if isinstance(coletanea, str) and coletanea[0] == '{':
        ids_colet.append(int(eval(coletanea)['id']))
    else:
        ids_colet.append(-1)
        
filmes['id_coletanea'] = ids_colet

In [ ]:
filmes

,id,title,homepage,adult,budget,orig_language,orig_title,overview,id_coletanea,runtime,popularity,revenue,release_date,status
0,862,Toy Story,http://toystory.disney.com/toy-story,False,30000000,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",10194,81.0,21.946943,373554033.0,1995-10-30,Released
1,8844,Jumanji,NaN,False,65000000,en,Jumanji,When siblings Judy and Peter discover an encha...,-1,104.0,17.015539,262797249.0,1995-12-15,Released
2,15602,Grumpier Old Men,NaN,False,0,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,119050,101.0,11.7129,0.0,1995-12-22,Released
3,31357,Waiting to Exhale,NaN,False,16000000,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",-1,127.0,3.859495,81452156.0,1995-12-22,Released
4,11862,Father of the Bride Part II,NaN,False,0,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,96871,106.0,8.387519,76578911.0,1995-02-10,Released
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,439050,Subdue,http://www.imdb.com/title/tt6209470/,False,0,fa,رگ خواب,Rising and falling between a man and woman.,-1,90.0,0.072051,0.0,NaN,Released
45462,111109,Century of Birthing,NaN,False,0,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,-1,360.0,0.178241,0.0,2011-11-17,Released
45463,67758,Betrayal,NaN,False,0,en,Betrayal,"When one of her hits goes wrong, a professiona...",-1,90.0,0.903007,0.0,2003-08-01,Released
45464,227506,Satan Triumphant,NaN,False,0,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",-1,87.0,0.003503,0.0,1917-10-21,Released


### Tabela coletânea 

In [ ]:
#coletaneasDf
idsColet = []
nomesColet = []
for linha in movies_metadata['belongs_to_collection']:
    if not isinstance(linha, str):
        continue
    x = eval(linha)
    if not isinstance(x, dict):
        continue
    id = x.get('id')
    if id not in idsColet:
        idsColet.append(id)
        nomesColet.append(x.get('name'))
            
coletaneas = {'id': idsColet, 'nome': nomesColet}
coletaneasDf = pd.DataFrame.from_dict(coletaneas)

In [ ]:
coletaneasDf

,id,nome
0,10194,Toy Story Collection
1,119050,Grumpy Old Men Collection
2,96871,Father of the Bride Collection
3,645,James Bond Collection
4,117693,Balto Collection
...,...,...
1690,104774,Tomtar och Trolltyg Collection
1691,400500,Чебурашка и крокодил Гена
1692,148603,Ducobu Collection
1693,152918,Mister Blot Collection


### Tabela Pessoa

In [ ]:
idPessoa = []
nomesPessoas = []
generosPessoas = []

idsVistos= set()

for indice, linha in credits.iterrows():
    elenco = eval(linha['cast'])
    equipe = eval(linha['crew'])
    for papel in elenco:
        if papel['id'] not in idsVistos:
            idsVistos.add(papel['id'])
            idPessoa.append(papel['id'])
            nomesPessoas.append(papel['name'])
            generosPessoas.append(papel['gender'])
    
    for membro in equipe:
        if membro['id'] not in idsVistos:
            idsVistos.add(membro['id'])
            idPessoa.append(membro['id'])
            nomesPessoas.append(membro['name'])
            generosPessoas.append(membro['gender'])


pessoas = {'id': idPessoa, 'nome': nomesPessoas, 'genero': generosPessoas}
pessoasDf = pd.DataFrame.from_dict(pessoas)

In [ ]:
pessoasDf

,id,nome,genero
0,31,Tom Hanks,2
1,12898,Tim Allen,2
2,7167,Don Rickles,2
3,12899,Jim Varney,2
4,12900,Wallace Shawn,2
...,...,...,...
353338,169498,Jeremy Lelliott,2
353339,1177500,Jason Widener,0
353340,93930,Kiko Ellsworth,2
353341,1261758,Aleksandr Chabrov,0


### Tabela Atuação

In [ ]:
#atuacoesDf
id_filmeAtuacoes = []
id_atorAtuacoes = []
nomesPersonagens = []

for indice, linha in credits.iterrows():
    elenco = eval(linha['cast'])
    for papel in elenco:
        id_atorAtuacoes.append(papel['id'])
        id_filmeAtuacoes.append(linha['id'])
        nomesPersonagens.append(papel['character'])

atuacoes = {'id_filme': id_filmeAtuacoes, 'id_ator': id_atorAtuacoes, 'personagem': nomesPersonagens}
atuacoesDf = pd.DataFrame.from_dict(atuacoes)

In [ ]:
atuacoesDf

,id_filme,id_ator,personagem
0,862,31,Woody (voice)
1,862,12898,Buzz Lightyear (voice)
2,862,7167,Mr. Potato Head (voice)
3,862,12899,Slinky Dog (voice)
4,862,12900,Rex (voice)
...,...,...,...
562469,227506,544742,
562470,227506,1090923,
562471,227506,1136422,
562472,227506,1261758,


### Tabela Producao

In [ ]:
#producoesDf
id_produtores = []
id_filmes = []
deptosProdutores = []
jobsProdutores = []

for indice, linha in credits.iterrows():
    elenco = eval(linha['crew'])
    for membro in elenco:
        id_produtores.append(membro['id'])
        id_filmes.append(linha['id'])
        deptosProdutores.append(membro['department'])
        jobsProdutores.append(membro['job'])
        
producoes = {'id_produtor': id_produtores, 'id_filme': id_filmes, 'job': jobsProdutores, 'department': deptosProdutores}
producoesDf = pd.DataFrame.from_dict(producoes)

In [ ]:
filmes

,id,title,homepage,adult,budget,original_language,original_title,overview,id_coletanea,runtime,popularity,revenue,release_date,status
0,862,Toy Story,http://toystory.disney.com/toy-story,False,30000000,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...","{'id': 10194, 'name': 'Toy Story Collection', ...",81.0,21.946943,373554033.0,1995-10-30,Released
1,8844,Jumanji,NaN,False,65000000,en,Jumanji,When siblings Judy and Peter discover an encha...,NaN,104.0,17.015539,262797249.0,1995-12-15,Released
2,15602,Grumpier Old Men,NaN,False,0,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",101.0,11.7129,0.0,1995-12-22,Released
3,31357,Waiting to Exhale,NaN,False,16000000,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",NaN,127.0,3.859495,81452156.0,1995-12-22,Released
4,11862,Father of the Bride Part II,NaN,False,0,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,"{'id': 96871, 'name': 'Father of the Bride Col...",106.0,8.387519,76578911.0,1995-02-10,Released
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,439050,Subdue,http://www.imdb.com/title/tt6209470/,False,0,fa,رگ خواب,Rising and falling between a man and woman.,NaN,90.0,0.072051,0.0,NaN,Released
45462,111109,Century of Birthing,NaN,False,0,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,NaN,360.0,0.178241,0.0,2011-11-17,Released
45463,67758,Betrayal,NaN,False,0,en,Betrayal,"When one of her hits goes wrong, a professiona...",NaN,90.0,0.903007,0.0,2003-08-01,Released
45464,227506,Satan Triumphant,NaN,False,0,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",NaN,87.0,0.003503,0.0,1917-10-21,Released


### Tabela Avaliacao

In [270]:
#avaliacoesDf
movie_ids = []
notas = []
timestamps = []
ids = []

i = 0
for indice, linha in ratings.iterrows():
    movie_ids.append(int(linha['movieId']))
    notas.append(linha['rating'])
    timestamps.append(int(linha['timestamp']))
    ids.append(i)
    i += 1
    if i > 200000:
        break

avaliacoes = {'id': ids, 'rating': notas, 'timestamp': timestamps, 'id_filme': movie_ids}
avaliacoesDf = pd.DataFrame.from_dict(avaliacoes)

In [272]:
avaliacoesDf

,id,rating,timestamp,id_filme
0,0,1.0,1425941529,110
1,1,4.5,1425942435,147
2,2,5.0,1425941523,858
3,3,5.0,1425941546,1221
4,4,5.0,1425941556,1246
...,...,...,...,...
199996,199996,4.0,844507298,208
199997,199997,3.0,844507253,231
199998,199998,2.0,844507324,253
199999,199999,1.0,844507324,288


### Tabela KeyWords

In [ ]:
id_filmes = []
palavras = []

for indice, linha in keywords.iterrows():
    objeto = eval(linha['keywords'])
    for palavra in objeto:
        id_filmes.append(linha['id'])
        palavras.append(palavra['name'])

keywords = {'id_movie': id_filmes, 'keyword': palavras}
keywordsDf = pd.DataFrame.from_dict(keywords)

In [ ]:
keywordsDf

,id_movie,keyword
0,862,jealousy
1,862,toy
2,862,boy
3,862,friendship
4,862,friends
...,...,...
158675,289923,mockumentary
158676,439050,tragic love
158677,111109,artist
158678,111109,play


### Tabela Genero

In [ ]:
#generosDf
genVistos = set()

nomesGeneros = []
idGeneros = []

for indice, linha in movies_metadata.iterrows():
    gens = eval(linha['genres'])
    for genero in gens:
        if genero['name'] in genVistos:
            continue
        genVistos.add(genero['name'])
        nomesGeneros.append(genero['name'])
        idGeneros.append(genero['id'])

generos = {'id': idGeneros, 'nome': nomesGeneros}
generosDf = pd.DataFrame.from_dict(generos)

In [ ]:
generosDf

,id,nome
0,16,Animation
1,35,Comedy
2,10751,Family
3,12,Adventure
4,14,Fantasy
5,10749,Romance
6,18,Drama
7,28,Action
8,80,Crime
9,53,Thriller


### Tabela genero_filme

In [ ]:
id_filmes = []
id_generos = []

for indice, linha in movies_metadata.iterrows():
    gens = eval(linha['genres'])
    for genero in gens:
        id_filmes.append(linha['id'])
        id_generos.append(genero['id'])

generoFilme = {'id_movie': id_filmes, 'id_genero': id_generos}
generoFilmeDf = pd.DataFrame.from_dict(generoFilme)

In [ ]:
generoFilmeDf

,id_movie,id_genero
0,862,16
1,862,35
2,862,10751
3,8844,12
4,8844,14
...,...,...
91101,439050,10751
91102,111109,18
91103,67758,28
91104,67758,18


### Popular BD

In [ ]:
filmes

,id,title,homepage,adult,budget,original_language,original_title,overview,id_coletanea,runtime,popularity,revenue,release_date,status
0,862,Toy Story,http://toystory.disney.com/toy-story,False,30000000,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",10194,81.0,21.946943,373554033.0,1995-10-30,Released
1,8844,Jumanji,NaN,False,65000000,en,Jumanji,When siblings Judy and Peter discover an encha...,-1,104.0,17.015539,262797249.0,1995-12-15,Released
2,15602,Grumpier Old Men,NaN,False,0,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,119050,101.0,11.7129,0.0,1995-12-22,Released
3,31357,Waiting to Exhale,NaN,False,16000000,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",-1,127.0,3.859495,81452156.0,1995-12-22,Released
4,11862,Father of the Bride Part II,NaN,False,0,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,96871,106.0,8.387519,76578911.0,1995-02-10,Released
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,439050,Subdue,http://www.imdb.com/title/tt6209470/,False,0,fa,رگ خواب,Rising and falling between a man and woman.,-1,90.0,0.072051,0.0,NaN,Released
45462,111109,Century of Birthing,NaN,False,0,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,-1,360.0,0.178241,0.0,2011-11-17,Released
45463,67758,Betrayal,NaN,False,0,en,Betrayal,"When one of her hits goes wrong, a professiona...",-1,90.0,0.903007,0.0,2003-08-01,Released
45464,227506,Satan Triumphant,NaN,False,0,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",-1,87.0,0.003503,0.0,1917-10-21,Released


In [ ]:
### Populando Genero
user = 'root'
passwd = 'password'
host =  'localhost'
port = 3306
database = 'Movies'
conn = pymysql.connect(host=host,
                       port=port,
                       user=user, 
                       passwd=passwd,  
                       db=database,
                       charset='utf8')

cursor = conn.cursor()

for indice, linha in generosDf.iterrows():
    query = f"insert into Genero (id, nome) values (%s, %s)"
    cursor.execute(query, (linha['id'], linha['nome']))

conn.commit()

conn.close()

In [ ]:
### Populando Coletanea
user = 'root'
passwd = 'password'
host =  'localhost'
port = 3306
database = 'Movies'
conn = pymysql.connect(host=host,
                       port=port,
                       user=user, 
                       passwd=passwd,  
                       db=database,
                       charset='utf8')

cursor = conn.cursor()

for indice, linha in coletaneasDf.iterrows():
    query = f"insert into Coletanea (id, nome) values (%s, %s)"
    cursor.execute(query, (linha['id'], linha['nome']))

conn.commit()

conn.close()

In [ ]:
filmes

,id,title,homepage,adult,budget,orig_language,orig_title,overview,id_coletanea,runtime,popularity,revenue,release_date,status
0,862,Toy Story,http://toystory.disney.com/toy-story,False,30000000,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",10194,81.0,21.946943,373554033.0,1995-10-30,Released
1,8844,Jumanji,NaN,False,65000000,en,Jumanji,When siblings Judy and Peter discover an encha...,-1,104.0,17.015539,262797249.0,1995-12-15,Released
2,15602,Grumpier Old Men,NaN,False,0,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,119050,101.0,11.7129,0.0,1995-12-22,Released
3,31357,Waiting to Exhale,NaN,False,16000000,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",-1,127.0,3.859495,81452156.0,1995-12-22,Released
4,11862,Father of the Bride Part II,NaN,False,0,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,96871,106.0,8.387519,76578911.0,1995-02-10,Released
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,439050,Subdue,http://www.imdb.com/title/tt6209470/,False,0,fa,رگ خواب,Rising and falling between a man and woman.,-1,90.0,0.072051,0.0,NaN,Released
45462,111109,Century of Birthing,NaN,False,0,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,-1,360.0,0.178241,0.0,2011-11-17,Released
45463,67758,Betrayal,NaN,False,0,en,Betrayal,"When one of her hits goes wrong, a professiona...",-1,90.0,0.903007,0.0,2003-08-01,Released
45464,227506,Satan Triumphant,NaN,False,0,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",-1,87.0,0.003503,0.0,1917-10-21,Released


In [288]:
### Populando Filmes

tabelas = {'Avaliacao': avaliacoesDf}
#            'Producao': producoesDf, 'Keywords': keywordsDf, 
#            'genero_filme': generoFilmeDf}

user = 'root'
passwd = 'password'
host =  'localhost'
port = 3306
database = 'Movies'
conn = pymysql.connect(host=host,
                       port=port,
                       user=user, 
                       passwd=passwd,  
                       db=database,
                       charset='utf8')

cursor = conn.cursor()

def validaData (string):
    for c in string:
        if not c.isdigit() and not c == '-':
            return False
    return True

idsVistos = set()

for indice, linha in filmes.iterrows():
    if linha['id'].isdigit():
        idsVistos.add(int(linha['id']))

for nomeTabela in tabelas:
    campos = "("
    for campo in tabelas[nomeTabela]:
        campos += campo
        campos += ', '
    campos = campos[:-2] + ') '

    formats = "("
    for i in tabelas[nomeTabela]:
        formats += r"%s, "
    formats = formats[:-2] + ')'

    for indice, linha in tabelas[nomeTabela].iterrows():
        atributos = []
        certo = True
        for campo in tabelas[nomeTabela]:
            if campo == "timestamp":
                atributos.append('2017-11-12 00:00:00')
            elif campo == 'id_filme' and linha[campo] not in idsVistos:
                certo = False
                break
            else:
                atributos.append(linha[campo])
        if certo:
            tupla = tuple(atributos)
            query = 'insert into ' + nomeTabela + ' ' + campos + 'values ' + formats
            cursor.execute(query, tupla)

conn.commit()

conn.close()

In [ ]:
#Atuacao         
#Avaliacao        
Coletanea        
#Filmes           
#Genero          
#Pessoa           
Producao         
genero_filme     
keywords